In [1]:
import tensorflow as tf
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


## 导入数据

In [2]:
import pickle

dataset = None

with open('../../common/xo_dataset.bin', 'rb') as f:
    dataset = pickle.load(f)
    
# 训练集图像向量
train_images = dataset['train_images']
# 训练集标签
train_labels = dataset['train_labels']
# 测试集图像向量
test_images = dataset['test_images']
# 测试集标签
test_labels = dataset['test_labels']

## 数据预处理

In [3]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## 生成Batch数据


In [4]:
def next_batch(train_images, train_labels, batch_size):
    idxs = np.array([i for i in range(len(train_images))])
    np.random.shuffle(idxs)
    top_idxs = idxs[:batch_size]
    return train_images[top_idxs], train_labels[top_idxs]
    

## 权重初始化

In [5]:

def weight_variable(shape):
    '''
    权重初始化  weight init
    初始化为一个接近0的很小的正数  init to a small number close to zero
    '''
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)



def bias_variable(shape):
    '''Bias变量初始化'''
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)


定义网络结构

In [6]:
# 输入的是28x28的图片 通道数为1
input_size = [None, 28, 28]
# Dropout的概率
keep_prob = tf.placeholder("float")

# 输入数据
x = tf.placeholder(tf.float32, input_size)
x_image = tf.reshape(x, [-1, 28, 28, 1])  

# Flatten层
x_flat = tf.layers.flatten(x_image)
# 第一层 全连接层1
W_fc1 = weight_variable([784, 128])
b_fc1 = bias_variable([128])
h_fc1 = tf.nn.relu(tf.matmul(x_flat, W_fc1) + b_fc1)
# Dropout
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 第二层 全连接层2
W_fc2 = weight_variable([128, 128])
b_fc2 = bias_variable([128])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)


# 第三层 Softmax输出层
W_fc3 = weight_variable([128, 2])
b_fc3 = bias_variable([2])
y_nn = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


训练和评估模型
ADAM优化器来做梯度最速下降,feed_dict中加入参数keep_prob控制dropout比例


In [7]:
# 真实的label数据 one_hot矩阵
y_ref = tf.placeholder("float", [None, 2])

In [8]:
# 交叉熵
cross_entropy = -tf.reduce_sum(y_ref * tf.log(y_nn))
learning_rate = 0.0001 # 学习率
# 定义优化器为ADam
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy) #使用adam优化器来以0.0001的学习率来进行微调
# 是否准确的布尔值列表
correct_prediction = tf.equal(tf.argmax(y_nn,1), tf.argmax(y_ref,1)) #判断预测标签和实际标签是否匹配
#　计算平均值即为准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

In [9]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer()) #初始化变量
merged = tf.summary.merge_all() 
writer = tf.summary.FileWriter('logs',sess.graph)


In [10]:
for i in range(500): #开始训练模型，循环训练5000次
    batch = next_batch(train_images, train_labels, 150) #batch大小设置为50
    if i % 100 == 0:
        train_accuracy = accuracy.eval(session = sess,
                                       feed_dict = {x:batch[0], y_ref:batch[1], keep_prob:1.0})
        print("step %d, train_accuracy %g" %(i, train_accuracy))
        # 保存检查点
        saver.save(sess, 'model/nn_xo.ckpt')
    train_step.run(session = sess, feed_dict = {x:batch[0], y_ref:batch[1],
                   keep_prob:0.7}) #神经元输出保持不变的概率 keep_prob 为0.5


step 0, train_accuracy 0.52
step 100, train_accuracy 0.993333
step 200, train_accuracy 1
step 300, train_accuracy 0.993333
step 400, train_accuracy 0.993333


In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
saver.restore(sess, 'model/nn_xo.ckpt')
# 查看测试集的准确率
print( "test accuracy %g" % accuracy.eval(feed_dict={
    x:test_images, y_ref:test_labels, keep_prob:1.0}))


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model/nn_xo.ckpt
test accuracy 0.99125


In [12]:

# 获取权重矩阵　偏移向量
WFC1 = W_fc1.eval()
BFC1 = b_fc1.eval()
WFC2 = W_fc2.eval()
BFC2 = b_fc2.eval()
WFC3 = W_fc3.eval()
BFC3 = b_fc3.eval()

In [13]:
# 创建一个新的图
# 变量改为常量

In [14]:
g = tf.Graph()

In [15]:
with g.as_default():
    x_image = tf.placeholder("float", shape=[None, 28, 28, 1], name="input_node")
    
    WFC1 = tf.constant(WFC1, name="WFC1")
    BFC1 = tf.constant(BFC1, name="BFC1")
    FC1 = tf.layers.flatten(x_image)
    FC1 = tf.nn.relu(tf.matmul(FC1, WFC1) + BFC1)
    
    WFC2 = tf.constant(WFC2, name="WFC2")
    BFC2 = tf.constant(BFC2, name="BFC2")
    FC2 = tf.nn.relu(tf.matmul(FC1, WFC2) + BFC2)
    
    WFC3 = tf.constant(WFC3, name="WFC3")
    BFC3 = tf.constant(BFC3, name="BFC3")
    OUTPUT = tf.nn.softmax(tf.matmul(FC2, WFC3) + BFC3, name="output_node")
    
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    graph_def = g.as_graph_def()
    tf.train.write_graph(graph_def, "./", "nn_xo.pb", as_text=False)
    